In [1]:
import tc_functions as fun
import pandas as pd

import importlib.util
spec = importlib.util.spec_from_file_location("hurdat", "../ORB/python/convective-structure/hurdat.py")
h = importlib.util.module_from_spec(spec)
spec.loader.exec_module(h)

In [2]:
# Import storms since 2015 from Kim's dataset that includes shortest distance to land, and do some maneuvering to make
# things closer to the HURDAT data straight from the website for east integration with the Hurdat() class (like column names, 
# naming convention for unnamed storms, and some additional columns)
nal_15 = pd.read_table("data/DTL_NAL_2015.txt", delimiter = ' ')
nal_16 = pd.read_table("data/DTL_NAL_2016.txt", delimiter = ' ')
nal_17 = pd.read_table("data/DTL_NAL_2017.txt", delimiter = ' ')
nal_18 = pd.read_table("data/DTL_NAL_2018.txt", delimiter = ' ')
nal_19 = pd.read_table("data/DTL_NAL_2019.txt", delimiter = ' ')
enp_15 = pd.read_table("data/DTL_ENP_2015.txt", delimiter = ' ')
enp_16 = pd.read_table("data/DTL_ENP_2016.txt", delimiter = ' ')
enp_17 = pd.read_table("data/DTL_ENP_2017.txt", delimiter = ' ')
enp_18 = pd.read_table("data/DTL_ENP_2018.txt", delimiter = ' ')
enp_19 = pd.read_table("data/DTL_ENP_2019.txt", delimiter = ' ')
total = pd.concat([nal_15, nal_16, nal_17, nal_18, nal_19, enp_15, enp_16, enp_17, enp_18, enp_19], ignore_index = True)

total.columns = ['YEAR', "PART_ID", "NAME", "DATETIME", "LAT", "LON", "WIND", "PRESSURE", "CATEGORY", "DISTANCE"]

total['DATETIME'] = pd.to_datetime(total['DATETIME'], format = "%Y%m%d%H")
total['NAME'] = total['NAME'].str.upper()
name_convert = {'UNNAMED04': 'FOUR', 
                'UNNAMED11': 'ELEVEN',
                'UNNAMED08': 'EIGHT'}
total['NAME'].loc[total['NAME'].str.contains('UNNAMED')] = [name_convert[x] for x in total['NAME'].loc[total['NAME'].str.contains('UNNAMED')]]
total['ID'] = total['PART_ID'] + total['YEAR'].astype(str)
hurdat_all = h.Hurdat(data = total)

In [3]:
# Filter storms by minimum intensity of 50
hurdat = hurdat_all.genesis_to_lysis_filter(minimum_wind = 50)
# Add RI / RW labels to all storm observations
hurdat.identify_events(threshold = 25)

In [4]:
# Next comes the fun part of taking each of the storms listed below and doing the following:

# FIRST: add the 250 km(? look up true dist) from land restriction and add as a column BEFORE calculating RI/RW (or have it done within the RI/RW function) so that RI/RW events are not marked true if within that range (or something like that)

# Pull GFS data
# Find true storm center
# Remove vorticity from the data (could do this within the tc_functions world).
# Calculate wind shear radial profiles for each observation.
# Figure out how to store the above wind shear profiles efficiently and smartly for all data.

# Think about turnng tc_functions into a class? We will see

In [5]:
hurdat.storms

,CATEGORY,LAT,LON,WIND,ID,NAME,DATETIME,RI,RW
0,SS,31.9,-77.3,50,AL012015,ANA,2015-05-09 00:00:00,False,False
1,TS,32.2,-77.5,50,AL012015,ANA,2015-05-09 06:00:00,False,False
2,TS,32.5,-77.8,50,AL012015,ANA,2015-05-09 12:00:00,False,False
3,TS,28.0,-96.0,50,AL022015,BILL,2015-06-16 12:00:00,False,False
4,TS,28.2,-96.7,50,AL022015,BILL,2015-06-16 18:00:00,False,False
...,...,...,...,...,...,...,...,...,...
2554,HU,22.6,-108.8,65,EP152019,LORENA,2019-09-20 12:00:00,False,False
2555,HU,23.0,-109.2,75,EP152019,LORENA,2019-09-20 18:00:00,False,False
2556,HU,23.7,-109.5,70,EP152019,LORENA,2019-09-21 00:00:00,False,False
2557,TS,24.2,-110.2,60,EP152019,LORENA,2019-09-21 06:00:00,False,False
